In [3]:
import numpy as np
import xarray as xr
from datetime import datetime as dt
import sys
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cm_xml_to_matplotlib as cm # Note: this python script must be located in the same folder as the current script
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
xr.open_dataset(dri+fn+'01'+'.'+varName+'.nc')

In [1]:
# ************************************************************************************************
# *************************** USER INPUTS ARE REQUIRED IN THIS CELL ******************************
# ************************************************************************************************


# Setup strings that will be concatenated together to form the path to the file

# List all the variables to be uploaded ****
varName = 'Z3BOT'

# List all the hindcast days to be loaded
hcastDayStr = [None,'01','02','03','04','05','06','07','08','09','10']#,'11','12','13','14','15','16','17','18','19','20']
hcastDay = [None,1,2,3,4,5,6,7,8,9,10]#,11,12,13,14,15,16,17,18,19,20]

# These are the base strings of the hindcast file paths/names
# dri = '/home/jlarson1/data/CAM_hindcasts/'  # Used for agron computer
dri = '/lss/research/agon-lab/CAM_hindcasts/tratl_cam5.3_ne30_rneale.globe/' # used for Hadley computer
fn = 'tratl_cam5.3_ne30_rneale.globe.fcast.day'

In [ ]:
dataImport3hr = [None] * len(hcastDayStr)

# Generate a file name and open each dataset
for jj, day in enumerate(hcastDayStr):
    if jj == 0: continue # Skip zero index so that the hindcast indexing is more intuitive (e.g. day 7 hindcast is indexed with a 7)
            
#     print(dri+fn+day+'.'+varName+'.nc')
    dataImport3hr[jj] = xr.open_dataset(dri+fn+day+'.'+varName+'.nc')

## Observation geopotential
# driPRECTObs = '/home/jlarson1/data/' # Used for agron
driGeopObs = '/lss/research/agon-lab/ERA5_ITCZ/geop/'
fnGeopObs  = 'geop_1979-2019_daily.nc'

dataImportObs = xr.open_dataset(driGeopObs+fnGeopObs)

# ************************************************************************************************
# ************************************************************************************************
# ************************************************************************************************

In [ ]:
# Time average the 3-hourly wind hindcast data to get daily wind data

# Create an empty array to hold the daily averaged data
dataImport = np.zeros([len(hcastDayStr), len(dataImport3hr[1].time)//8, len(dataImport3hr[1].lat), len(dataImport3hr[1].lon)])

# iterate for the number of hindcast days
for ii in range(1, len(dataImport)):
    
    # Iterate for the number of time stamps in 3hrly data. Count by 8
    for jj in range(0, len(dataImport[1])*8, 8):
        dataImport[ii][jj//8] = dataImport3hr[ii]['Z3'][jj:jj+8].mean(dim='time').values

In [ ]:
# Convert hindcast time stamps from cftime to datetime format
# This is preferred due to the ease of working with datetime relative to cftime

# Initialize an array of time period (152 days) for each hindcast day (20 days)
dateTime = np.array([ [None] * (len(dataImport3hr[1].time)//8) ] * len(dataImport[0]))

# Loop through all hindcast days (should be 20 days)
# Start the index at 1 so the indexing is more intuitive (e.g. day 7 hindcast is indexed with a 7)
for ii in range(1,  len(dataImport) ):
    
    # Loop through each day in the hindcast dataset (should be 152 days)
    for jj in range(0, len(dataImport3hr[1]['time']), 8):
        
        # Parse through the time stamp and convert to the datetime class
        # The index inside the str() below is as follows: variable (precip)->hcastDay->time dim->specific time stamp->raw value
        dateTime[ii][jj//8] = dt.strptime(str(dataImport3hr[ii]['time'][jj+7].values), '%Y-%m-%d %H:%M:%S')

In [ ]:
# Recreate a new dataset for the daily averaged PGA data

# Initialize an empty array to hold the new dataArray
dataPGA = [None] * len(hcastDayStr)

# Loop through each hindcast day. Skip the zero (empty) index.
for ii in range(1, len(hcastDayStr)):
    
    # For the time index, take the time indexes from the original data import, convert them to datetime objects,
    # select every 8th index (one per day) and then keep only the date portion of the time stamp (no associated hour)
    dataPGA[ii] = xr.DataArray(dataImport[ii], dims='time', 'lat', 'lon'], 
                               coords={'time':dateTime[ii], 'lat':dataImport3hr[1]['lat'], 'lon':dataImport3hr})
#     dataPGA[ii] = xr.DataArray(dataImport[ii], dims=['time','lat','lon'], 
#                                coords={'time':dataImport3hr[ii].indexes["time"].to_datetimeindex()[7::8].date, 
#                                        'lat':dataImport3hr[1].lat, 'lon':dataImport3hr[1].lon})

In [ ]:
dataImport3hr[ii].indexes["time"].to_datetimeindex()[::8]

In [ ]:
# ************************************************************************************************
# *************************** USER INPUTS ARE REQUIRED IN THIS CELL ******************************
# ************************************************************************************************

#### Select the date in 'YYYY-MM-DD' format (ISO 8601) ####
_dayOfTheSeason = '2010-02-23'

#### Select the number of days to be time averaged ####
numDayAvg = 6

#### Select which day hindcast should be used ####
DayXXHcast = [1,5,10]  # Must be a list of integer(s) from >=1 and <=10 (max of 6 integers)

# Choose region of interest 
lats = -10
latn =  10
lonw = -135 # *** For longitudes west of the Prime Meridian, enter a negative number ***
lone = -90  

# User inputs no longer required 

# ************************************************************************************************
# ************************************************************************************************
# ************************************************************************************************

# Convert _dayOfTheSeason to datetime64 format
dayOfTheSeason = np.datetime64(_dayOfTheSeason)

In [ ]:
dataPGA[10].sel(time=slice(dayOfTheSeason, dayOfTheSeason+numDayAvg-1))

In [ ]:
## Select the data within the user defined region of interest

# First, check if the user has requested data that is out of the time bounds of hindcast dataset
# If so, null the variable index to prevent the script from running any further
if dayOfTheSeason+numDayAvg-1 > dataPGA[1]['time'][-1] or dayOfTheSeason < dataPGA[-1]['time'][0]:
    sys.exit("The user has attemped to access data that is outside of the time bounds of the hindcast dataset")
    
# Initialize empty array
selData = [None] * len(hcastDayStr)

# Iterate through each hindcast day and select the data in the region of interest
# The iteration starts at 1 to agree with the indexing convention of the hindcast days (starting with 1)
for ii in range(1, len(hcastDayStr)):
    # Subtract 1 from numDayAvg since a slice will include the end point
    # Add 360 to the longitude so that it agrees with the hindcast dataset lon convention (0-360)
    selData[ii] = dataPGA[ii].sel(time=slice(dayOfTheSeason, dayOfTheSeason+numDayAvg-1), 
                                   lat=slice(lats-0.6, latn+0.6), lon=slice(lonw+360, lone+360))
    
# Do the same for the obs
selDataObs = dataImportObs.sel(time=slice(dayOfTheSeason, dayOfTheSeason+numDayAvg), 
                               latitude=slice(latn, lats), longitude=slice(lonw+360, lone+360))

In [ ]:
# Zonally and temporally average the data for the specified length of numDayAvg

# Initialize lists for hindcast data
plotData = [None] * len(selData)
plotMap  = [None] * len(selData)

# Perform time and longitudinal average for all hindcast days
for ii in range(1, len(selData)):
    plotMap[ii]  = selData[ii].mean(dim='time')
    plotData[ii] = selData[ii].mean(dim=['lon','time'])
    
# Now for obs
# ALso, divide by gravity to convert from geopotential height to just geopotential
plotDataObs = selDataObs.mean(dim=['longitude','time']).z/9.80665
plotMapObs  = selDataObs.mean(dim='time').z/9.80665

In [ ]:
# Plot a comparison between hindcasts and observation

# Set pixel density of plot
_dpi = 100

# Setup the figure
fig = plt.figure(figsize=(13, 7.2), tight_layout='true')
plt.minorticks_on()
# ax=fig.add_axes()
plt.tick_params(axis='both', which='major', length=10, width=1.5, direction='in', labelsize=16, right='True', top='True')
plt.tick_params(axis='both', which='minor', length= 5, width=1.0, direction='in', labelsize=16, right='True', top='True')
plt.tick_params(axis='x', labelsize=16)
plt.tick_params(axis='y', labelsize=16)
plt.axhline(0, color='0', linewidth = 1.5, zorder=-1)
plt.ylabel('latitude $(^{\circ})$', fontsize=16)
plotColors = ['#C8102E','#F1BE48','#524727','#9B945F','#8B5B29', '#BE531C'] # Go Cyclones! :)
plotLegend = []
# plotLegend.append('TRMM')

# plt.xlim([,])
plt.xlabel('geopotential height [m]', fontsize=16)


# Loop through the DayXXHcast list to overlay multiple days of hindcast data
for ii, DayXX in enumerate(DayXXHcast):
    
    p1 = plt.plot(plotData[DayXX], plotData[DayXX].lat, color = plotColors[ii], label = 'Day '+str(DayXX)) # linewidth = 2.5,    

p2 = plt.plot(plotDataObs, plotDataObs['latitude'],  color='#006BA6', linestyle='dashed', label='ERA5 reanalysis')
plt.title("Hindcast "+varName+" vs observation for "+str(dayOfTheSeason)+" and a "+str(numDayAvg)+
          " day average - Longitudinally averaged from ("+str(lonw)+','+str(lone)+'E)')
plt.legend()

# # Save figure code
# # Check if folder is there. If it is not, make the folder
# dir = '/home/jlarson1/'+str(dayOfTheSeason)+' Analysis'
# if os.path.isdir(dir) == False:
#     os.mkdir(dir)

# plt.savefig(dir+'/'+str(dayOfTheSeason)+'with'+str(numDayAvg)+'DayAvg'+varName+'LinePlot', bbox_inches='tight', dpi=_dpi)
    
plt.show()

In [ ]:
# Create a map plot of the observation data

##### Control plotting variables here #####

# Set maximum of colorbar [mm/day]
Vmax = 30

# # Set density of arrows with n. A greater integer reduces the density of arrows
# n = 8

# Set the density of the pixels in the figure. This also affects the size of the saved figure
_dpi = 200

###########################################

# Create the figure and apply asthetic constraints
fig = plt.figure(figsize=(17, 6)) 

# Prepare colormap
mycmap = cm.make_cmap('LarsonColormap.xml')

geo_axes = plt.axes(projection=ccrs.PlateCarree())
geo_axes.set_xticks(np.arange(lonw,lone+1,15), crs=ccrs.PlateCarree())
# Use user specified lat values for tick marks
geo_axes.set_yticks(np.arange(lats, latn+1,5), crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
geo_axes.xaxis.set_major_formatter(lon_formatter)
geo_axes.yaxis.set_major_formatter(lat_formatter)
geo_axes.coastlines(resolution='110m')
geo_axes.gridlines(xlocs=np.arange(lonw,lone+1,15), ylocs=np.arange(lats, latn+1,5))
plt.axhline(0, color='0', linewidth = 1.5)
image_extent = [lonw,lone,lats,latn]

# Plot precip contour plot
IM = geo_axes.contourf(plotMapObs.longitude, plotMapObs.latitude, plotMapObs, levels=200, transform=ccrs.PlateCarree(),
                       cmap=plt.get_cmap(mycmap))

# Set boundaries of plot
geo_axes.set_xlim([lonw,lone])
geo_axes.set_ylim([lats,latn])

plt.colorbar(IM, ax=geo_axes, orientation='vertical', pad=0.02, extend='both')
plt.title("map plot of ERA5 geopotential height [m] for "+str(dayOfTheSeason)+" and a "+str(numDayAvg)+" Day Average")
    
plt.show()

In [ ]:
# Plot four subplots for each hindcast day

##### Control plotting variables here #####

# Set which days are plotted
hcastDay = [5,10]
# hcastDay = [1,2,3,4]

# Set the max value of the colorbar
Vmax = 70

# Set the density of the wind vector arrows. A greater integer reduces the density
n = int(3)

###########################################

# Set border of precip data
image_extent = [lonw,lone,lats,latn]

# Pre-generate the subplots with a PlateCarree projection
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(11, 9), subplot_kw=dict(projection=ccrs.PlateCarree()))

# Prepare colormap
mycmap = cm.make_cmap('LarsonColormap.xml')

# Iterate through the subplots
for ii, day in enumerate(hcastDay):
    
    # Plot precip in mm/day
#     im = axes.flat[ii].contourf(plotMap[day].lon, plotMap[day].lat, plotMap[day].values, levels=200, 
#                                 transform=ccrs.PlateCarree(), cmap=plt.get_cmap(mycmap), extend='both', vmax=Vmax)
    im = axes.flat[ii].imshow(plotMap[day].values, extent=image_extent, vmax=Vmax, origin='lower', 
                              cmap=plt.get_cmap(mycmap))
    
    # Set boundaries of graphs
    axes.flat[ii].set_xlim([lonw,lone])
    axes.flat[ii].set_ylim([lats,latn]) # Use user specified lat values
    
    # Set asthetic constraints of the plot
    axes.flat[ii].set_xticks(np.arange(lonw,lone+1,15), crs=ccrs.PlateCarree())
    axes.flat[ii].set_yticks(np.arange(lats,latn+1,5),  crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    axes.flat[ii].xaxis.set_major_formatter(lon_formatter)
    axes.flat[ii].yaxis.set_major_formatter(lat_formatter)
    axes.flat[ii].coastlines(resolution='110m')
    axes.flat[ii].gridlines(xlocs=np.arange(lonw,lone+1,15), ylocs=np.arange(lats,latn+1,5)) 
    axes.flat[ii].title.set_text('day '+str(day)+' hindcast')
    axes.flat[ii].axhline(0, color='0', linewidth = 1.5)
    

# The following code creates a new axes for the colorbar
fig.subplots_adjust(right=0.825) #0.825
# Add base axis for color bar. Located at 85% from left and 15% from bottom. Width is 2% of figure. Height is 70% of figure.
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7]) 
fig.colorbar(im, cax=cbar_ax, extend='both')

# Set title for the entire figure
fig.suptitle("hindcast geopotential height [m] for "+str(dayOfTheSeason)+" and a "+str(numDayAvg)+" day average",
             size=16,y=0.95)

plt.show()

In [ ]:
# Create a gradient of the geopotential for obs and hindcasts



In [ ]:
# Currently saving this code for a rainy day

#############################################################
# # Plot four subplots for each hindcast day

# ##### Control plotting variables here #####

# # Set which days are plotted
# hcastDay = [5,10]

# # Set the max value of the colorbar
# Vmax = 70

# # Set the density of the wind vector arrows. A greater integer reduces the density
# n = int(3)

# ###########################################


# # Pre-generate the subplots with a PlateCarree projection
# fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(11, 8.5), subplot_kw=dict(projection=ccrs.PlateCarree()))

# # Prepare colormap
# mycmap = cm.make_cmap('LarsonColormap.xml')

# # Iterate through the subplots
# for ii, day in enumerate(hcastDay):
    
#     # Plot precip in mm/day
    
#     im = axes.flat[ii].contourf(plotMap[day].lon, plotMap[day].lat, plotMap[day].values, levels=200, 
#                                 transform=ccrs.PlateCarree(), cmap=plt.get_cmap(mycmap), vmax=Vmax)

#     # Set boundaries of graphs
#     axes.flat[ii].set_xlim([lonw,lone])
#     axes.flat[ii].set_ylim([lats,latn]) # Use user specified lat values
    
#     # Set asthetic constraints of the plot
#     axes.flat[ii].set_xticks(np.arange(lonw,lone+1,15), crs=ccrs.PlateCarree())
#     axes.flat[ii].set_yticks(np.arange(lats,latn+1,5),  crs=ccrs.PlateCarree())
#     lon_formatter = LongitudeFormatter(zero_direction_label=True)
#     lat_formatter = LatitudeFormatter()
#     axes.flat[ii].xaxis.set_major_formatter(lon_formatter)
#     axes.flat[ii].yaxis.set_major_formatter(lat_formatter)
#     axes.flat[ii].coastlines(resolution='110m')
#     axes.flat[ii].gridlines(xlocs=np.arange(lonw,lone+1,15), ylocs=np.arange(lats,latn+1,5)) 
#     axes.flat[ii].title.set_text('day '+str(day)+' hindcast')
#     axes.flat[ii].axhline(0, color='0', linewidth = 1.5)
    
# # The following code creates a new axes for the colorbar
# fig.subplots_adjust(right=0.825) #0.825
# # Add base axis for color bar. Located at 85% from left and 15% from bottom. Width is 2% of figure. Height is 70% of figure.
# cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
# fig.colorbar(im, cax=cbar_ax, extend='both')

# # Set title for the entire figure
# fig.suptitle("hindcast geopotential height [m] for "+str(dayOfTheSeason)+" and a "+str(numDayAvg)+" day average",
#              size=16,y=0.95)

# plt.show()